In [5]:
import pandas as pd

df=pd.read_csv('..\\data\\fraud.csv', index_col = 0)
y = df['Class'].values
df = df.iloc[:,1:]
X = df.drop(columns = 'Class').values

print(df.shape)
print(df.head(5))

(21693, 30)
         V1        V2        V3        V4        V5        V6        V7  \
0  1.176563  0.323798  0.536927  1.047002 -0.368652 -0.728586  0.084678   
1  0.681109 -3.934776 -3.801827 -1.147468 -0.735540 -0.501097  1.038865   
2  1.140729  0.453484  0.247010  2.383132  0.343287  0.432804  0.093380   
3 -1.107073 -3.298902 -0.184092 -1.795744  2.137564 -1.684992 -2.015606   
4 -0.314818  0.866839 -0.124577 -0.627638  2.651762  3.428128  0.194637   

         V8        V9       V10  ...       V21       V22       V23       V24  \
0 -0.069246 -0.266389  0.155315  ... -0.109627 -0.341365  0.057845  0.499180   
1 -0.626979 -2.274423  1.527782  ...  0.652202  0.272684 -0.982151  0.165900   
2  0.173310 -0.808999  0.775436  ... -0.003802  0.058556 -0.121177 -0.304215   
3 -0.007181 -0.165760  0.869659  ...  0.130648  0.329445  0.927656 -0.049560   
4  0.670674 -0.442658  0.133499  ... -0.312774 -0.799494 -0.064488  0.953062   

        V25       V26       V27       V28  Amount  Class

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                    test_size = 0.40, 
                                    random_state=1)
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

Shape of X_train: (13015, 29)
Shape of X_test: (8678, 29)


In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

#--------------------------------------------------
## ------------Logistic Regresion----------------## 
#--------------------------------------------------

from sklearn.linear_model import LogisticRegression

steps = [('scaler', StandardScaler()),         
         ('logReg', LogisticRegression(penalty = "l1", C =1, solver='liblinear'))]

LR_pipeline = Pipeline(steps)
LR_pipeline.fit(X_train, y_train)

,steps,"[('scaler', ...), ('logReg', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,penalty,'l1'
,dual,False
,tol,0.0001
,C,1


In [8]:
#--------------------------------------------------
## Model Evaluation ## 
#--------------------------------------------------
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score


ypred_test = LR_pipeline.predict(X_test)
mat_clf = confusion_matrix(y_test, ypred_test)
report_clf = classification_report(y_test, ypred_test)

print(mat_clf)
print(report_clf)

ypred_testP = LR_pipeline.predict_proba(X_test)
auc = roc_auc_score(y_test, ypred_testP[:,1])
print(auc)

[[8535    6]
 [  27  110]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8541
           1       0.95      0.80      0.87       137

    accuracy                           1.00      8678
   macro avg       0.97      0.90      0.93      8678
weighted avg       1.00      1.00      1.00      8678

0.9828846175211539


In [9]:
ypred_train = LR_pipeline.predict(X_train)
mat_clf = confusion_matrix(y_train, ypred_train)
report_clf = classification_report(y_train, ypred_train)

print(mat_clf)
print(report_clf)

ypred_trainP = LR_pipeline.predict_proba(X_train)
auc = roc_auc_score(y_train, ypred_trainP[:,1])
print(auc)

[[12793     3]
 [   46   173]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12796
           1       0.98      0.79      0.88       219

    accuracy                           1.00     13015
   macro avg       0.99      0.89      0.94     13015
weighted avg       1.00      1.00      1.00     13015

0.9796718723459528


In [11]:
# Linear regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

steps = [
    ('scaler', StandardScaler()),
    ('linReg', LinearRegression())
]
LIN_pipeline = Pipeline(steps)
LIN_pipeline.fit(X_train, y_train)

,steps,"[('scaler', ...), ('linReg', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None


In [12]:
# Predictions
ypred_test_lin = LIN_pipeline.predict(X_test)
ypred_test_lin_class = (ypred_test_lin >= 0.5).astype(int)

# Metrics
print("**Linear Regression Results**")
print(confusion_matrix(y_test, ypred_test_lin_class))
print(classification_report(y_test, ypred_test_lin_class))
print("AUC:", roc_auc_score(y_test, ypred_test_lin))

**Linear Regression Results**
[[8540    1]
 [  60   77]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      8541
           1       0.99      0.56      0.72       137

    accuracy                           0.99      8678
   macro avg       0.99      0.78      0.86      8678
weighted avg       0.99      0.99      0.99      8678

AUC: 0.9763356997633571


In [13]:
# Polynomial regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

steps = [
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2)),
    ('linReg', LinearRegression())
]
POLY_pipeline = Pipeline(steps)
POLY_pipeline.fit(X_train, y_train)

,steps,"[('scaler', ...), ('poly', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,degree,2
,interaction_only,False
,include_bias,True
,order,'C'


In [14]:
# Predictions
ypred_test_poly = POLY_pipeline.predict(X_test)
ypred_test_poly_class = (ypred_test_poly >= 0.5).astype(int)

# Metrics
print("**Polynomial Regression Results**")
print(confusion_matrix(y_test, ypred_test_poly_class))
print(classification_report(y_test, ypred_test_poly_class))
print("AUC:", roc_auc_score(y_test, ypred_test_poly))

**Polynomial Regression Results**
[[8535    6]
 [  25  112]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8541
           1       0.95      0.82      0.88       137

    accuracy                           1.00      8678
   macro avg       0.97      0.91      0.94      8678
weighted avg       1.00      1.00      1.00      8678

AUC: 0.9652658665757358
